# PROJET PYTHON POUR LA DATA SCIENCE 2022

# THEME : ANALYSE DES AVIS DES STAGIAIRES ET EMPLOYES DE SOCIETE GENERALE TRANSMIS VIA LA PLATEFORME INDEED.COM ENTRE 2012 ET 2022  

## Auteurs:
* TALEB AHMED Raja
* MAGAJIE WAMSA Berthe Magella
* RENE Léo

## ANALYSE DE DONNEES ET NATURAL LANGUAGE PROCESSING (NLP)

### INTRODUCTION

  ### Contexte et Justification du projet

  Société Générale est l'un des tout premiers groupes européens de services financiers et acteur important de l’économie depuis plus de 150 ans. Au quotidien, 25 millions de clients sont accompagnés grâce à ses plus de 117 000 collaborateurs répartis dans 66 pays. Le groupe Société Générale se positionne en partenaire de confiance pour ses clients, engagé dans le développement durable de l’Afrique, ou bien encore pour l'accompagnement de la transition écologique mondiale. Quatre principales valeurs y sont partagées selon le site officiel du groupe : Esprit d’équipe, Innovation, Responsabilité, Engagement (https://www.societegenerale.com/fr/le-groupe-societe-generale/identite/presentation).  
  
  L’ENSAE Paris a noué des partenariats structurants avec des entreprises visant à améliorer leur visibilité auprès des élèves et à favoriser l’émergence des projets communs. Société Générale fait partie de ces partenaires prestigeux. Avec plus de 600 métiers dans le monde, l’entreprise recrute en effet des talents dans de nombreux secteurs : risques, IT, data, consulting interne, etc. Pour les étudiants ou jeunes diplômés curieux et motivés par les défis d’une grande banque, il s'agirait du cadre adéquat; aussi, sa promesse est d'offrir à tout employé un environnement dans lequel il s'épanouie en développant son potentiel (https://www.ensae.fr/entreprises/devenir-partenaire-de-lensae-paris/nos-partenaires).
  
  Indeed est une plateforme de recherche d'emplois qui permet aux employeurs de publier des offres et aux personnes en quête d'emplois de postuler; aussi des avis sur les entreprises y sont transmis, faisant office de retour d'expérience. Ces avis, s'ils sont bien exploités, peuvent être utiles non seulement aux chercheurs d'emplois dans leur quête, mais aussi aux entreprises pour l'amélioration du quotidien de leurs employés. 

  ### Objectifs
  
L'objectif général de ce projet est d'évaluer la Société Générale au travers des retours d'expériences et avis de ceux qui y ont travaillé et ceux qui y travaillent encore. Plus spécifiquement, il s'agit de:
   * Décrire les avis transmis sur Indeed selon plusieurs critères;
   * Présenter un niveau d'appréciation global;
   * Ressortir les préoccupations les plus fréquentes;
   * Analyser les potentiels facteurs qui influencent le niveau d'appréciation des employés et stagiaires. 

  ### Méthodologie du projet

Afin d'atteindre les objectifs énoncés ci-dessus, nous avons utilisé les méthodes suivantes:

   - Web-Scrapping :
Pour constituer notre base de données, nous avons extrait des données structurées sur le Web de manière automatisée; autrement dit, nous avons usé du webscrapping sur le site Indeed.com.

   - NLP (Natural Language Processing):
C'est une méthode à l’intersection du Machine Learning et de la linguistique ayant pour but d'extraire de l'information et une signification d’un contenu textuel. Nous l'avons utilisée ici pour effectuer de la detection des langues dans les commentaires, les wordclouds, l'analyse sentimentale ainsi que la detection des différentes aspects les plus abordés dans les commentaires (topic modeling).

In [1]:
# Importation des packages utiles
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn

### TRAITEMENT DES DONNEES

In [13]:
# Importation de la base de données
df = pd.read_csv("Database.csv",sep=",")

In [14]:
# Description de la base de données initiale
print("Il y a", len(df), "individus dans notre base de données.")
print(df.columns)
df.sample(3)

Il y a 2520 individus dans notre base de données.
Index(['Country', 'rating', 'contract', 'is_employed', 'job', 'city', 'date',
       'comment'],
      dtype='object')


,Country,rating,contract,is_employed,job,city,date,comment
1741,France,5.0,NaN,False,Juriste au sein du Service Juridique,"Clichy, A8",2013,Une expérience enrichissante professionnelleme...
585,France,5.0,NaN,False,Auxiliaire de vacances,Marly-le-Roi (78),2019,Emploi formateur intéressant humainement et va...
1590,France,5.0,NaN,False,Administrateur MEGA,La Défense,2015,Support applicatif niveau 3:\nPROJET : Adminis...


In [15]:
#Création d'une copie de la base
df_new = df.copy()

In [16]:
#Remplacer True et False dans la  colonne is_employed du DataFrame par 1 et 0
df_new['is_employed'] = df_new['is_employed'].replace([True], 1)
df_new['is_employed'] = df_new['is_employed'].replace([False], 0)

In [17]:
#Vérification types des variables
print(df_new.dtypes)

Country         object
rating         float64
contract        object
is_employed      int64
job             object
city            object
date             int64
comment         object
dtype: object


In [18]:
#Nombre de modalités pour chacune des variables
df_new.nunique()

Country           3
rating            5
contract          3
is_employed       2
job            1706
city            803
date             11
comment        2518
dtype: int64

In [19]:
#Vérification valeurs manquantes
df_new.count()

Country        2520
rating         2520
contract        237
is_employed    2520
job            2520
city           2520
date           2520
comment        2520
dtype: int64

La seule variable contenant des valeurs manquantes est "contract" avec seulement 237 valeurs renseignées; Un peu moins du dizième des personnes ont précisé le contrat dans leurs commentaires.

In [20]:
#Suppression des codes postaux dans la colonne city 
characters = ")(1234567890"
df_new['city'] = df['city'].apply(lambda x: x.strip(characters))
df_new.head()

,Country,rating,contract,is_employed,job,city,date,comment
0,France,3.0,cdd,0,Conseillère clientèle,Massy,2022,"Bonne équipe, dommage que la direction soit bl..."
1,France,5.0,NaN,0,Responsable,Falaise,2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,France,4.0,NaN,0,Compliance Officer,Nanterre,2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,France,5.0,NaN,0,Assistante administrative,Saint-Dié-des-Vosges,2022,"Missions variées, solidarité dans l’équipe et ..."
4,France,4.0,NaN,0,Chargée de communication,La Défense,2022,Première expérience professionnelle durant 3 a...


In [21]:
#Ajustement de la police pour éviter les problèmes liés à la sensibilité à la casse.
df_new['city'] = df_new['city'].apply(lambda x: x.lower())
df_new['comment']=df_new['comment'].apply(lambda x: x.lower())
df_new['job']=df_new['job'].apply(lambda x: x.lower())
df_new.head()

,Country,rating,contract,is_employed,job,city,date,comment
0,France,3.0,cdd,0,conseillère clientèle,massy,2022,"bonne équipe, dommage que la direction soit bl..."
1,France,5.0,NaN,0,responsable,falaise,2022,"équipe au top, l'esprit d'équipe fait la force..."
2,France,4.0,NaN,0,compliance officer,nanterre,2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,France,5.0,NaN,0,assistante administrative,saint-dié-des-vosges,2022,"missions variées, solidarité dans l’équipe et ..."
4,France,4.0,NaN,0,chargée de communication,la défense,2022,première expérience professionnelle durant 3 a...


In [22]:
# Définition d'une fonction qui permettra de nettoyer nos données textuelles
import re
def clean(texte):
    # Suppression des liens
    for item in re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+").findall(texte):
        texte=texte.replace(item, "")
    # Suppression du retour à la ligne
    texte = texte.replace("\n", " ").replace("\r", "") 
    # Suppression des caractères ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\+\*\µ,\:\&\§]", " ", texte) 
    # Suppression des apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # Suppression des points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # Suppression des adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # Suppression des mots contenant des chiffres
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    # Diviser les mots attachés
    texte = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",texte) if s])
    return texte

In [23]:
df_new["clean_comment"] = df_new["comment"].apply(lambda x: clean(x))
df_new["clean_city"] = df_new["city"].apply(lambda x: clean(x))
df_new["clean_job"] = df_new["job"].apply(lambda x: clean(x))
df_new.head()

,Country,rating,contract,is_employed,job,city,date,comment,clean_comment,clean_city,clean_job
0,France,3.0,cdd,0,conseillère clientèle,massy,2022,"bonne équipe, dommage que la direction soit bl...",bonne équipe dommage que la direction soit bl...,massy,conseillère clientèle
1,France,5.0,NaN,0,responsable,falaise,2022,"équipe au top, l'esprit d'équipe fait la force...",équipe au top l esprit d équipe fait la force...,falaise,responsable
2,France,4.0,NaN,0,compliance officer,nanterre,2022,"bonne ambiance, bonne équipe, bon relationnel,...",bonne ambiance bonne équipe bon relationnel ...,nanterre,compliance officer
3,France,5.0,NaN,0,assistante administrative,saint-dié-des-vosges,2022,"missions variées, solidarité dans l’équipe et ...",missions variées solidarité dans l équipe et ...,saint-dié-des-vosges,assistante administrative
4,France,4.0,NaN,0,chargée de communication,la défense,2022,première expérience professionnelle durant 3 a...,première expérience professionnelle durant an...,la défense,chargée de communication


### ANALYSE DESCRIPTIVE DES DONNEES

Notre base de données contient 2520 lignes qui représentent les individus et 8 colonnes qui sont nos variables que sont : **'country'** qui renferme le pays concerné (France, US ou Inde), **'rating'** qui renferme les notes attribuées (sur 5), **'contract'** pour le type de contrat(Stage, cdd, cdi),  **'is_employed'** qui prend la valeur 1 si l'individu travaille au sein de la SG au moment du commentaire et 0 sinon, **'job'** pour le poste occupé, **'city'** pour le lieu de travail (la ville) , **'date'** pour l'année de publication du commentaire et **'comment'** pour les commentaires.

In [24]:
df_new['rating'].describe()

count    2520.000000
mean        3.806746
std         1.089283
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

La plus petite note attribuée est 1 et la plus grande est 5 (Evaluation sur 5). En moyenne, la SG est appréciée par ceux qui y ont travaillé (ou qui y travaillent) avec une note de  de 3,8.B 

In [25]:
#Effectifs pour chaque modalité de la variable rating 
df_new['rating'].value_counts()

4.0    985
5.0    738
3.0    501
2.0    164
1.0    132
Name: rating, dtype: int64

In [26]:
#Effectifs pour chaque modalité de la variable contract 
print(df_new['contract'].count())
df_new['contract'].value_counts()

237


stage    210
cdd       21
cdi        6
Name: contract, dtype: int64

Seuls 237 individus de notre base ont précisé leur contrat de travail au sein de la SG et ce sont en grande majorité des stagiaires.

In [27]:
#Effectifs pour chaque modalité de la variable is_employed
df_new['is_employed'].value_counts()

0    1885
1     635
Name: is_employed, dtype: int64

Les individus ne travaillant plus au sein de la SG au moment du commentaire sont trois fois plus nombreux que ceux en emploi à ce moment.

### VISUALISATION DES DONNEES

### NATURAL LANGUAGE PROCESSING (NLP)

In [28]:
#Installation des packages nécessaires

!pip install langdetect 
!pip install wordcloud
!pip install nltk
!pip install  gensim
!pip install vaderSentiment
!pip install textblob
!pip install vaderSentiment-fr
!pip install deep-translator
!pip install yellowbrick

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=17deb7c9328cd044ddbc60e7936fe9a5df65c46a9d4d10395a95c56e005e8aa3
  Stored in directory: /home/onyxia/.cache/pip/wheels/c4/16/af/1889804d8b7c0c041cadee8e29673a938a332acbf2865c70a1
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 11.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 15.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 23.0 MB/s eta 0:00:00
  Created wheel for vaderSentiment-fr: filename=vaderSentiment_fr-1.3.4-py3-none-any.whl size=185962 sha25

In [29]:
#Importation des librairies nécessaires

from langdetect import detect # pour détecter les langues
import sklearn as sk
from wordcloud import WordCloud # pour gérer les nuages de mots
from deep_translator import GoogleTranslator 

#stopword français
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords') 

# stopword anglais
import gensim 
from gensim.parsing.preprocessing import STOPWORDS 

#from gensim import corpora

from PIL import Image # pour personaliser la forme du wordcloud

# Pour l'analyse des sentiments
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SenEN
analyzer = SenEN()

#Pour les classements des mots les plus fréquents
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text.freqdist import FreqDistVisualizer
from yellowbrick.style import set_palette

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/onyxia/nltk_data...


#### Détection des langues

In [30]:
# Fonction pour détecter les langues utilisées
def language_detection(text):
    try:
        return detect(text)
    except:
        return None


In [31]:
df_new['langues']=df_new['comment'].apply(language_detection)
print(f"Les commentaires sont rédigés en {len(df_new.langues.unique())} langues")
df_new['langues'].value_counts()

Les commentaires sont rédigés en 6 langues


fr    1738
en     772
es       5
pt       3
tr       1
da       1
Name: langues, dtype: int64